In [24]:
import tensorflow as tf
from tensorflow import lite
from tensorflow.keras import models

In [ ]:
best_model_1_filename = 'models/best_model_1_fold_10.h5'
best_model_2_filename = 'models/best_model_2_fold_10.h5'
best_model_3_filename = 'models/best_model_3_fold_4.h5'
best_model_4_filename = 'models/best_model_4_fold_2.h5'
best_model_5_filename = 'models/best_model_5_fold_4.h5'
best_model_6_filename = 'models/best_model_6_fold_10.h5'
best_model_7_filename = 'models/best_model_7_fold_7.h5'
best_model_8_filename = 'models/best_model_8_fold_6.h5'
best_model_9_filename = 'models/best_model_9_fold_3.h5'
best_model_10_filename = 'models/best_model_10_fold_2.h5'

In [2]:
# Parameters
keras_model_filename = 'models/best_model_1_fold_10.h5'
tflite_filename = 'wake_word_stop_lite_1.tflite'
c_model_name = 'wake_word_stop_1_h' 

In [5]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [22]:
import psutil

# Provera trenutne upotrebe memorije
print(f"Current memory usage: {psutil.virtual_memory()}")

Current memory usage: svmem(total=16701485056, available=6082228224, percent=63.6, used=10084945920, free=323686400, active=11170242560, inactive=3501506560, buffers=215302144, cached=6077550592, shared=185769984, slab=850468864)


In [20]:
import gc

# Pokretanje Garbage Collector-a
gc.collect()

0

In [21]:
import tensorflow as tf

# Oslobađanje memorije TensorFlow sesije
tf.keras.backend.clear_session()

In [28]:
import tensorflow as tf
from tensorflow import lite
from tensorflow.keras import models
import os
from pathlib import Path

# Parameters
keras_model_filename = 'models/best_model_10_fold_2.h5'
tflite_filename = 'wake_word_stop_lite_10.tflite'
c_model_name = 'wake_word_stop_10_h'

try:
    # Proverite i kreirajte direktorijum ako ne postoji
    keras_model_path = Path(keras_model_filename)
    if not keras_model_path.is_file():
        raise FileNotFoundError(f"Keras model file '{keras_model_filename}' not found.")

    # Učitajte Keras model
    print(f"Loading Keras model from {keras_model_filename}")
    model = models.load_model(keras_model_filename, custom_objects={'precision_m': precision_m, 'recall_m': recall_m, 'f1_m': f1_m})

    # Konvertujte model u TF Lite format
    print("Converting Keras model to TF Lite format...")
    converter = lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # converter.target_spec.supported_types = [tf.float16]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    # converter.allow_custom_ops=True
    tflite_model = converter.convert()
    print("Conversion successful.")

    # Proverite i kreirajte direktorijum za čuvanje konvertovanog modela
    tflite_model_path = Path(tflite_filename)
    tflite_model_path.parent.mkdir(parents=True, exist_ok=True)

    # Sačuvajte konvertovani TF Lite model
    print(f"Saving TF Lite model to {tflite_filename}")
    with open(tflite_filename, 'wb') as f:
        f.write(tflite_model)

    print(f"TF Lite model is saved at {tflite_filename}")

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Loading Keras model from models/best_model_10_fold_2.h5
Converting Keras model to TF Lite format...
INFO:tensorflow:Assets written to: /tmp/tmp6rzu6d0d/assets


INFO:tensorflow:Assets written to: /tmp/tmp6rzu6d0d/assets


Conversion successful.
Saving TF Lite model to wake_word_stop_lite_10.tflite
TF Lite model is saved at wake_word_stop_lite_10.tflite


2024-06-12 23:25:12.110221: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-12 23:25:12.110242: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-12 23:25:12.110406: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp6rzu6d0d
2024-06-12 23:25:12.111550: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-12 23:25:12.111560: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp6rzu6d0d
2024-06-12 23:25:12.114604: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-12 23:25:12.158876: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp6rzu6d0d
2024-06-12 23:25:12.173990: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 63585 m

In [ ]:
# Convert model to TF Lite model
model = models.load_model(keras_model_filename)
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.allow_custom_ops=True
tflite_model = converter.convert()
open(tflite_filename, 'wb').write(tflite_model)

In [29]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [46]:
tflite_filename = 'wake_word_stop_lite_10.tflite'
c_model_name = 'wake_word_stop_10_h'

In [47]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))

In [29]:
# Function #2: converting to tfLite
import numpy as np
import os

def convert_tflite_to_header(tflite_path, output_header_path):

    with open(tflite_path, 'rb') as tflite_file:
        tflite_content = tflite_file.read()
    
    
    hex_lines = [', '.join([f'0x{byte:02x}' for byte in tflite_content[i:i+12]]) for i in range(0, len(tflite_content), 12)]
    
    
    hex_array = ',\n  '.join(hex_lines)
    
    
    with open(output_header_path, 'w') as header_file:
        
        header_file.write('const unsigned char model[] = {\n  ')
        header_file.write(f'{hex_array}\n')
        header_file.write('};\n\n')
        
if __name__ == "__main__":
    tflite_path = 'wake_word_stop_lite.tflite'
    output_header_path = '2_wake_word_stop_h.h'

    convert_tflite_to_header(tflite_path, output_header_path)